tiktoken으로 토큰을 계산하는 방법

[`tiktoken`](https://github.com/openai/tiktoken/blob/main/README.md)은 OpenAI의 빠른 오픈 소스 토크나이저입니다.

텍스트 문자열(예: `"tiktoken is great!"`)과 인코딩(예: `"cl100k_base"`)이 주어지면 토크나이저는 텍스트 문자열을 토큰 목록(예: `["t", "ik", "token", "is", "great", "!"]`).

GPT 모델은 텍스트를 토큰 형태로 보기 때문에 텍스트 문자열을 토큰으로 분할하는 것이 유용합니다. 텍스트 문자열에 있는 토큰 수를 알면 (a) 텍스트 모델이 처리하기에 문자열이 너무 긴지 여부와 (b) OpenAI API 호출 비용(토큰별로 사용량이 책정됨)을 알 수 있습니다.

## 인코딩

인코딩은 텍스트가 토큰으로 변환되는 방식을 지정합니다. 다른 모델은 다른 인코딩을 사용합니다.

`tiktoken`은 OpenAI 모델에서 사용되는 세 가지 인코딩을 지원합니다.:

| Encoding name           | OpenAI models                                       |
|-------------------------|-----------------------------------------------------|
| `cl100k_base`           | `gpt-4`, `gpt-3.5-turbo`, `text-embedding-ada-002`  |
| `p50k_base`             | Codex models, `text-davinci-002`, `text-davinci-003`|
| `r50k_base` (or `gpt2`) | GPT-3 models like `davinci`                         |

다음과 같이 `tiktoken.encoding_for_model()`을 사용하여 모델의 인코딩을 검색할 수 있습니다.:
```python
encoding = tiktoken.encoding_for_model('gpt-3.5-turbo')
```

`p50k_base`는 `r50k_base`와 상당히 겹치며 코드가 아닌 애플리케이션의 경우 일반적으로 동일한 토큰을 제공합니다.

## 랭귀지별 토크나이저 라이브러리

For `cl100k_base` and `p50k_base` encodings:
- Python: [tiktoken](https://github.com/openai/tiktoken/blob/main/README.md)
- .NET / C#: [SharpToken](https://github.com/dmitry-brazhenko/SharpToken)

For `r50k_base` (`gpt2`) encodings, tokenizers are available in many languages.
- Python: [tiktoken](https://github.com/openai/tiktoken/blob/main/README.md) (or alternatively [GPT2TokenizerFast](https://huggingface.co/docs/transformers/model_doc/gpt2#transformers.GPT2TokenizerFast))
- JavaScript: [gpt-3-encoder](https://www.npmjs.com/package/gpt-3-encoder)
- .NET / C#: [GPT Tokenizer](https://github.com/dluc/openai-tools)
- Java: [gpt2-tokenizer-java](https://github.com/hyunwoongko/gpt2-tokenizer-java)
- PHP: [GPT-3-Encoder-PHP](https://github.com/CodeRevolutionPlugins/GPT-3-Encoder-PHP)

(OpenAI는 타사 라이브러리를 보증하거나 보증하지 않습니다.)

## 문자열이 일반적으로 토큰화되는 방식

영어에서 토큰의 길이는 일반적으로 한 문자에서 한 단어(예: `"t"` 또는 `"great"`)이지만 일부 언어에서는 토큰이 한 문자보다 짧거나 한 단어보다 길 수 있습니다. 공백은 일반적으로 단어의 시작으로 그룹화됩니다(예: `"is "` 또는 `" "`+`"is"` 대신 `" is"`). [OpenAI Tokenizer](https://beta.openai.com/tokenizer)에서 문자열이 어떻게 토큰화되는지 빠르게 확인할 수 있습니다.

## 0. Install `tiktoken`

필요한 경우 `pip`와 함께 `tiktoken`을 설치합니다.:

In [1]:
%pip install --upgrade tiktoken

Note: you may need to restart the kernel to use updated packages.


## 1. Import `tiktoken`

In [1]:
import tiktoken


## 2. Load an encoding

`tiktoken.get_encoding()`을 사용하여 이름으로 인코딩을 로드하십시오.

처음 실행할 때 다운로드하려면 인터넷 연결이 필요합니다. 나중에 실행할 때 인터넷 연결이 필요하지 않습니다.

In [2]:
encoding = tiktoken.get_encoding("cl100k_base")


주어진 모델 이름에 대한 올바른 인코딩을 자동으로 로드하려면 `tiktoken.encoding_for_model()`을 사용하십시오.

In [3]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

## 3. Turn text into tokens with `encoding.encode()`



`.encode()` 메서드는 텍스트 문자열을 토큰 정수 목록으로 변환합니다.

In [4]:
encoding.encode("tiktoken is great!")


[83, 1609, 5963, 374, 2294, 0]

`.encode()`에 의해 반환된 목록의 길이를 계산하여 토큰을 계산합니다.

In [5]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens


In [6]:
num_tokens_from_string("tiktoken is great!", "cl100k_base")


6

## 4. Turn tokens into text with `encoding.decode()`

`.decode()`는 토큰 정수 목록을 문자열로 변환합니다.

In [7]:
encoding.decode([83, 1609, 5963, 374, 2294, 0])


'tiktoken is great!'

경고: `.decode()`는 단일 토큰에 적용될 수 있지만 utf-8 경계에 있지 않은 토큰의 경우 손실이 발생할 수 있습니다.

단일 토큰의 경우 `.decode_single_token_bytes()`는 단일 정수 토큰을 그것이 나타내는 바이트로 안전하게 변환합니다.

In [8]:
[encoding.decode_single_token_bytes(token) for token in [83, 1609, 5963, 374, 2294, 0]]


[b't', b'ik', b'token', b' is', b' great', b'!']

(문자열 앞의 `b`는 문자열이 바이트 문자열임을 나타냅니다.)

## 5. Comparing encodings

다른 인코딩은 단어를 분할하고, 공백을 그룹화하고, 영어가 아닌 문자를 처리하는 방법에 따라 다릅니다. 위의 방법을 사용하여 몇 가지 예제 문자열에서 서로 다른 인코딩을 비교할 수 있습니다.

In [9]:
def compare_encodings(example_string: str) -> None:
    """Prints a comparison of three string encodings."""
    # print the example string
    print(f'\nExample string: "{example_string}"')
    # for each encoding, print the # of tokens, the token integers, and the token bytes
    for encoding_name in ["gpt2", "p50k_base", "cl100k_base"]:
        encoding = tiktoken.get_encoding(encoding_name)
        token_integers = encoding.encode(example_string)
        num_tokens = len(token_integers)
        token_bytes = [encoding.decode_single_token_bytes(token) for token in token_integers]
        print()
        print(f"{encoding_name}: {num_tokens} tokens")
        print(f"token integers: {token_integers}")
        print(f"token bytes: {token_bytes}")
        

In [10]:
compare_encodings("antidisestablishmentarianism")



Example string: "antidisestablishmentarianism"

gpt2: 5 tokens
token integers: [415, 29207, 44390, 3699, 1042]
token bytes: [b'ant', b'idis', b'establishment', b'arian', b'ism']

p50k_base: 5 tokens
token integers: [415, 29207, 44390, 3699, 1042]
token bytes: [b'ant', b'idis', b'establishment', b'arian', b'ism']

cl100k_base: 6 tokens
token integers: [519, 85342, 34500, 479, 8997, 2191]
token bytes: [b'ant', b'idis', b'establish', b'ment', b'arian', b'ism']


In [11]:
compare_encodings("2 + 2 = 4")



Example string: "2 + 2 = 4"

gpt2: 5 tokens
token integers: [17, 1343, 362, 796, 604]
token bytes: [b'2', b' +', b' 2', b' =', b' 4']

p50k_base: 5 tokens
token integers: [17, 1343, 362, 796, 604]
token bytes: [b'2', b' +', b' 2', b' =', b' 4']

cl100k_base: 7 tokens
token integers: [17, 489, 220, 17, 284, 220, 19]
token bytes: [b'2', b' +', b' ', b'2', b' =', b' ', b'4']


In [12]:
compare_encodings("お誕生日おめでとう")



Example string: "お誕生日おめでとう"

gpt2: 14 tokens
token integers: [2515, 232, 45739, 243, 37955, 33768, 98, 2515, 232, 1792, 223, 30640, 30201, 29557]
token bytes: [b'\xe3\x81', b'\x8a', b'\xe8\xaa', b'\x95', b'\xe7\x94\x9f', b'\xe6\x97', b'\xa5', b'\xe3\x81', b'\x8a', b'\xe3\x82', b'\x81', b'\xe3\x81\xa7', b'\xe3\x81\xa8', b'\xe3\x81\x86']

p50k_base: 14 tokens
token integers: [2515, 232, 45739, 243, 37955, 33768, 98, 2515, 232, 1792, 223, 30640, 30201, 29557]
token bytes: [b'\xe3\x81', b'\x8a', b'\xe8\xaa', b'\x95', b'\xe7\x94\x9f', b'\xe6\x97', b'\xa5', b'\xe3\x81', b'\x8a', b'\xe3\x82', b'\x81', b'\xe3\x81\xa7', b'\xe3\x81\xa8', b'\xe3\x81\x86']

cl100k_base: 9 tokens
token integers: [33334, 45918, 243, 21990, 9080, 33334, 62004, 16556, 78699]
token bytes: [b'\xe3\x81\x8a', b'\xe8\xaa', b'\x95', b'\xe7\x94\x9f', b'\xe6\x97\xa5', b'\xe3\x81\x8a', b'\xe3\x82\x81', b'\xe3\x81\xa7', b'\xe3\x81\xa8\xe3\x81\x86']


## 6. Counting tokens for chat API calls

`gpt-3.5-turbo` 및 `gpt-4`와 같은 ChatGPT 모델은 이전 완성 모델과 동일한 방식으로 토큰을 사용하지만 메시지 기반 형식으로 인해 대화에서 사용할 토큰 수를 계산하기가 더 어렵습니다. .

다음은 `gpt-3.5-turbo-0301` 또는 `gpt-4-0314`로 전달된 메시지의 토큰을 계산하는 함수의 예입니다.

메시지에서 토큰을 계산하는 정확한 방법은 모델마다 다를 수 있습니다. 시간을 초월한 보증이 아니라 추정치 아래의 함수에서 카운트를 고려하십시오.

In [13]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo":
        print("Warning: gpt-3.5-turbo may change over time. Returning num tokens assuming gpt-3.5-turbo-0301.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301")
    elif model == "gpt-4":
        print("Warning: gpt-4 may change over time. Returning num tokens assuming gpt-4-0314.")
        return num_tokens_from_messages(messages, model="gpt-4-0314")
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif model == "gpt-4-0314":
        tokens_per_message = 3
        tokens_per_name = 1
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens


In [14]:
# let's verify the function above matches the OpenAI API response

import openai

example_messages = [
    {
        "role": "system",
        "content": "You are a helpful, pattern-following assistant that translates corporate jargon into plain English.",
    },
    {
        "role": "system",
        "name": "example_user",
        "content": "New synergies will help drive top-line growth.",
    },
    {
        "role": "system",
        "name": "example_assistant",
        "content": "Things working well together will increase revenue.",
    },
    {
        "role": "system",
        "name": "example_user",
        "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage.",
    },
    {
        "role": "system",
        "name": "example_assistant",
        "content": "Let's talk later when we're less busy about how to do better.",
    },
    {
        "role": "user",
        "content": "This late pivot means we don't have time to boil the ocean for the client deliverable.",
    },
]

for model in ["gpt-3.5-turbo-0301", "gpt-4-0314"]:
    print(model)
    # example token count from the function defined above
    print(f"{num_tokens_from_messages(example_messages, model)} prompt tokens counted by num_tokens_from_messages().")
    # example token count from the OpenAI API
    response = openai.ChatCompletion.create(
        model=model,
        messages=example_messages,
        temperature=0,
        max_tokens=1  # we're only counting input tokens here, so let's not waste tokens on the output
    )
    print(f'{response["usage"]["prompt_tokens"]} prompt tokens counted by the OpenAI API.')
    print()


gpt-3.5-turbo-0301
127 prompt tokens counted by num_tokens_from_messages().
127 prompt tokens counted by the OpenAI API.

gpt-4-0314
129 prompt tokens counted by num_tokens_from_messages().
129 prompt tokens counted by the OpenAI API.

